In [10]:
import os
import numpy as np
import tensorflow as tf
from collections import deque

def sample(logits):
    noise1 = tf.random_uniform(tf.shape(logits))
#     noise2 = tf.random_uniform(tf.shape(logits))
    # the values should be random in the beginning, so should be different
    # in the beginning before returning usually the same actions for both
    # what if it chooses the 'do-nothing action?'
    # maybe just use sequence of actions as different output (8 actions = 8 output)
    # maybe don't use this framework at all and do the alternate classifier training ( seperate 
    # usual cnns) and reinforcement learning.
    return tf.argmax(logits - tf.log(-tf.log(3*noise1)), 1)#,
#             tf.argmax(logits - tf.log(-tf.log(noise2)), 1)


def fc(x, scope, nh, act=tf.nn.relu, init_scale=1.0):
    with tf.variable_scope(scope):
        nin = x.get_shape()[1].value
        w = tf.get_variable("w", [nin, nh], initializer=ortho_init(init_scale))
        b = tf.get_variable("b", [nh], initializer=tf.constant_initializer(0.0))
        z = tf.matmul(x, w)+b
        h = act(z)
        return h
    
def ortho_init(scale=1.0):
    def _ortho_init(shape, dtype, partition_info=None):
        #lasagne ortho init for tf
        shape = tuple(shape)
        if len(shape) == 2:
            flat_shape = shape
        elif len(shape) == 4: # assumes NHWC
            flat_shape = (np.prod(shape[:-1]), shape[-1])
        else:
            raise NotImplementedError
        a = np.random.normal(0.0, 1.0, flat_shape)
        u, _, v = np.linalg.svd(a, full_matrices=False)
        q = u if u.shape == flat_shape else v # pick the one with the correct shape
        q = q.reshape(shape)
        return (scale * q[:shape[0], :shape[1]]).astype(np.float32)
    return _ortho_init

def cat_entropy(logits):
    a0 = logits - tf.reduce_max(logits, 1, keepdims=True)
    ea0 = tf.exp(a0)
    z0 = tf.reduce_sum(ea0, 1, keepdims=True)
    p0 = ea0 / z0
    return tf.reduce_sum(p0 * (tf.log(z0) - a0), 1)

# def cat_entropy_softmax(p0):
#     return - tf.reduce_sum(p0 * tf.log(p0 + 1e-6), axis = 1)

def mse(pred, target):
    return tf.square(pred-target)/2.

def find_trainable_variables(key):
    with tf.variable_scope(key):
        return tf.trainable_variables()

def discount_with_dones(rewards, dones, gamma):
    discounted = []
    r = 0
    for reward, done in zip(rewards[::-1], dones[::-1]):
        r = reward + gamma * r * (1. - done)  # fixed off by one bug
        discounted.append(r)
    return discounted[::-1]

def make_path(f):
    return os.makedirs(f, exist_ok=True)

def constant(p):
    return 1

def linear(p):
    return 1-p

def my_explained_variance(qpred, q):
    _, vary = tf.nn.moments(q, axes=[0, 1])
    _, varpred = tf.nn.moments(q - qpred, axes=[0, 1])
    check_shape([vary, varpred], [[]] * 2)
    return 1.0 - (varpred / vary)

In [70]:
for i in range(10):
    noise = tf.random_uniform(tf.shape((2,3,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2)))
    with tf.Session() as sess:  print(tf.argmax((2,8,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2) - tf.log(-tf.log(4*noise)), -1).eval()) 
    # noise[0].val
    sess.close()

3
25
1
1
1
20
7
2
13
24


In [76]:
A = np.array([41, 162, 349, 641, 531, 445, 35])
B = np.array([42, 300, 323, 479, 436, 389, 36])
C = np.array([1.4, 7, 14, 28, 70, 140])
D = np.where(A==41, A, B)
D

array([ 41, 300, 323, 479, 436, 389,  36])